In [ ]:
import os
from glob import glob
import cv2
import open3d as o3d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)

In [ ]:
DATA_PATH = r'2011_09_26/2011_09_26_drive_0106_sync'

left_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_02/data/*.png')))
right_image_paths = sorted(glob(os.path.join(DATA_PATH, 'image_03/data/*.png')))

# get LiDAR data
bin_paths = sorted(glob(os.path.join(DATA_PATH, 'velodyne_points/data/*.bin')))

print(f"Number of left images: {len(left_image_paths)}")
print(f"Number of right images: {len(right_image_paths)}")
print(f"Number of LiDAR point clouds: {len(bin_paths)}")

In [ ]:
with open(r'2011_09_26/calib_cam_to_cam.txt','r') as f:
    calib = f.readlines()

# get projection matrices
P_left = np.array([float(x) for x in calib[25].strip().split(' ')[1:]]).reshape((3,4))
P_right = np.array([float(x) for x in calib[33].strip().split(' ')[1:]]).reshape((3,4))

# get rectified rotation matrices
R_left_rect = np.array([float(x) for x in calib[24].strip().split(' ')[1:]]).reshape((3, 3,))
R_right_rect = np.array([float(x) for x in calib[32].strip().split(' ')[1:]]).reshape((3, 3,))

R_left_rect = np.insert(R_left_rect, 3, values=[0,0,0], axis=0)
R_left_rect = np.insert(R_left_rect, 3, values=[0,0,0,1], axis=1)

In [ ]:
def decompose_projection_matrix(P):    
    K, R, T, _, _, _, _ = cv2.decomposeProjectionMatrix(P)
    T = T/T[3]

    return K, R, T

In [ ]:
K_left, R_left, T_left = decompose_projection_matrix(P_left)
K_right, R_right, T_right = decompose_projection_matrix(P_right)

In [ ]:
with open(r'2011_09_26/calib_velo_to_cam.txt', 'r') as f:
    calib = f.readlines()

R_cam_velo = np.array([float(x) for x in calib[1].strip().split(' ')[1:]]).reshape((3, 3))
t_cam_velo = np.array([float(x) for x in calib[2].strip().split(' ')[1:]])[:, None]

T_cam_velo = np.vstack((np.hstack((R_cam_velo, t_cam_velo)),
                        np.array([0, 0, 0, 1])))

T_cam_velo

In [ ]:
index = 10

left_image = cv2.cvtColor(cv2.imread(left_image_paths[index]), cv2.COLOR_BGR2RGB)
right_image = cv2.cvtColor(cv2.imread(right_image_paths[index]), cv2.COLOR_BGR2RGB)

In [ ]:
print(f"The shape of an image is: {left_image.shape}")

In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 15))
ax1.imshow(left_image)
ax1.set_title('Left Image', size=22)
ax2.imshow(right_image)
ax2.set_title('Right Image', size=22);

In [ ]:
def get_velo2cam(lidar_bin):
    ''' Converts the LiDAR point cloud to camera (u, v, z) image coordinates, 
        where z is in meters
        '''
    # read in LiDAR data
    scan_data = np.fromfile(lidar_bin, dtype=np.float32).reshape((-1,4))

    # convert to homogeneous coordinate system
    velo_points = scan_data[:, 0:3] # (x, y, z) --> (front, left, up)
    velo_points = np.insert(velo_points, 3, 1, axis=1).T # homogeneous LiDAR points

    # delete negative liDAR points
    velo_points = np.delete(velo_points, np.where(velo_points[3, :] < 0), axis=1) 

    # possibly use RANSAC to remove the ground plane for better viewing?

    # convert to camera coordinates
    velo_camera = P_left @ R_left_rect @ T_cam_velo @ velo_points

    # delete negative camera points ??
    velo_camera  = np.delete(velo_camera , np.where(velo_camera [2,:] < 0)[0], axis=1) 

    # get camera coordinates u,v,z
    velo_camera[:2] /= velo_camera[2, :]

    return velo_camera

In [ ]:
def project_velo2cam(lidar_bin, image):
    ''' Projects LiDAR point cloud onto the image coordinate frame '''

    # get camera (u, v, z) coordinates
    velo_camera = get_velo2cam(lidar_bin)

    (u, v, z) = velo_camera

    # remove outliers (points outside of the image frame)
    img_h, img_w, _ = image.shape
    u_out = np.logical_or(u < 0, u > img_w)
    v_out = np.logical_or(v < 0, v > img_h)
    outlier = np.logical_or(u_out, v_out)
    velo_camera = np.delete(velo_camera, np.where(outlier), axis=1)
    
    return velo_camera

In [ ]:
lidar_bin = bin_paths[index]
(u, v, z) = project_velo2cam(lidar_bin, left_image)

In [ ]:
plt.imshow(left_image)
plt.scatter([u], [v], c=[z], cmap='rainbow_r', alpha=0.5, s=2);

In [ ]:
def compute_sgbm_disparity(left_image, right_image, num_disparities=5*16, 
                           block_size=11, window_size=5, display=False):
    """ Computes the disparity of an image pair using the SGBM algoithm.
        Inputs: 
            image_left/_right - (MxN) grayscale input images
            see opencv documentation for "StereoBM_create"
        Outputs:
            disparity (MxN) computed disparity map for the input images
        
        NOTE: image_left must be the left image (same for the right) or 
              unexpected results will occur due to 
    """
    # P1 and P2 control disparity smoothness (recommended values below)
    P1 = 8 * 3 * window_size**2
    P2 = 32 * 3 * window_size**2
    sgbm_obj = cv2.StereoSGBM_create(0, num_disparities, block_size, 
        P1, P2, mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY)
        
    # compute disparity
    disparity = sgbm_obj.compute(left_image, right_image).astype(np.float32)/16.0

    # display is desired
    if display:
      plt.figure(figsize = (40,20))
      plt.imshow(disparity, cmap='cividis')
      plt.title('Disparity Map', size=25)
      plt.show();

    return disparity

def calc_depth_map(disp_left, K_left, T_left, T_right):
    ''' Computes Depth map from Intrinsic Camera Matrix and Translations vectors.
        For KITTI, the depth is in meters.
        '''
    # Get the focal length from the K matrix
    f = K_left[0, 0]
    
    # Get the distance between the cameras from the t matrices (baseline)
    b = np.abs(T_left[0] - T_right[0])[0]
    
    # Replace all instances of 0 and -1 disparity with a small minimum value (to avoid div by 0 or negatives)
    disp_left[disp_left <= 0] = 1e-5
    
    # Calculate the depths 
    depth_map = f*b / disp_left 

    return depth_map

In [ ]:
left_image_gray = cv2.cvtColor(left_image, cv2.COLOR_RGB2GRAY)
right_image_gray = cv2.cvtColor(right_image, cv2.COLOR_RGB2GRAY)

In [ ]:
from IPython.display import Image
from ipywidgets import interact, interactive, fixed

disparity = interactive(compute_sgbm_disparity, 
                        left_image=fixed(left_image_gray), 
                        right_image=fixed(right_image_gray), 
                        num_disparities=(0,512,16), 
                        block_size=(1,19,2), 
                        window_size=(1,13,2),
                        display=fixed(True))
display(disparity)

In [ ]:
num_disparities = disparity.kwargs['num_disparities']
block_size = disparity.kwargs['block_size']
window_size = disparity.kwargs['window_size']

In [ ]:
disparity = compute_sgbm_disparity(left_image_gray, 
                                   right_image_gray, 
                                   num_disparities, 
                                   block_size, 
                                   window_size, 
                                   display=False)

In [ ]:
plt.imshow(disparity)

In [ ]:
depth_map = calc_depth_map(disparity, K_left, T_left, T_right)
plt.imshow(np.log(depth_map), cmap='rainbow_r'); # or 'cividis_r' or 'cividis'
# plt.imshow(np.log(np.log(depth_map)), cmap='viridis_r');
plt.scatter([u], [v], c=[z], cmap='rainbow_r', alpha=0.5, s=2);

In [ ]:
depth_map.shape
u.shape
v.shape
z.shape

def uvz_to_depth_map(uvz, image_shape):
    depth_map = np.zeros(image_shape, dtype=np.float32)
    
    u, v, z = uvz
    u, v = u.astype(np.int32), v.astype(np.int32)
    
    valid_indices = (u >= 0) & (u < image_shape[1]) & (v >= 0) & (v < image_shape[0])
    u, v, z = u[valid_indices], v[valid_indices], z[valid_indices]
    
    depth_map[v, u] = z
    
    return depth_map
    
lidar_depth_map = uvz_to_depth_map((u, v, z), depth_map.shape)
lidar_depth_map.shape

In [ ]:
plt.imshow(lidar_depth_map, cmap='rainbow_r')

In [ ]:
# Convert the depth map to Open3D depth image
depth_image_o3d = o3d.geometry.Image(np.log(depth_map).astype(np.float32))

# Create an Intrinsics object using camera parameters
height, width = depth_map.shape
fx, fy = K_left[0, 0], K_left[1, 1]
cx, cy = K_left[0, 2], K_left[1, 2]
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

# Create a point cloud from the depth image
point_cloud = o3d.geometry.PointCloud.create_from_depth_image(depth_image_o3d, intrinsic)

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
def depth_map_to_point_cloud(depth_map, K):
    """
    Convert a depth map into a 3D point cloud.

    :param depth_map: The depth map to convert.
    :param K: The intrinsic camera matrix.
    :return: Open3D point cloud object.
    """
    # Get the shape of the depth map
    height, width = depth_map.shape

    # Create a grid of coordinates corresponding to the depth map pixel indices
    x, y = np.meshgrid(np.arange(width), np.arange(height))

    # Normalize x and y to camera coordinates
    x_normalized = (x - K[0, 2]) / K[0, 0]
    y_normalized = (y - K[1, 2]) / K[1, 1]

    # Create the point cloud in camera coordinates
    points = np.stack((x_normalized * depth_map, y_normalized * depth_map, depth_map), axis=-1)

    # Reshape to a list of 3D points
    points = points.reshape(-1, 3)

    # Remove points where the depth is zero
    points = points[depth_map.flatten() > 0]

    # Create an Open3D point cloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    return pcd

In [ ]:
def apply_rainbow_color(pcd):
    """
    Apply a rainbow color map to a point cloud based on the z-coordinate.
    
    :param pcd: Open3D point cloud object.
    :return: Colored Open3D point cloud object.
    """
    # Convert Open3D point cloud to numpy array
    points = np.asarray(pcd.points)

    # Normalize z coordinates to the range [0, 1]
    z = points[:, 2]
    z_normalized = (z - np.min(z)) / (np.max(z) - np.min(z))

    # Apply a colormap (rainbow)
    colors = plt.cm.rainbow(z_normalized)[:, :3]

    # Assign colors to point cloud
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def calculate_point_colors_based_on_distance(point_cloud):
    # Calculate distances from the origin (0, 0, 0)
    distances = np.linalg.norm(point_cloud[:, :3], axis=1)

    # Map distances to colors (rainbow colormap)
    max_distance = np.max(distances)
    colors = plt.get_cmap("rainbow")(distances / max_distance)[:, :3]
    return colors

In [ ]:
def read_lidar_data(bin_path):
    # read in LiDAR data
    scan_data = np.fromfile(lidar_bin, dtype=np.float32).reshape((-1,4))

    # convert to homogeneous coordinate system
    velo_points = scan_data[:, 0:3] # (x, y, z) --> (front, left, up)
    
    # delete negative liDAR points
    velo_points = np.delete(velo_points, np.where(velo_points[3, :] < 0), axis=1) 

    # Convert to Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(velo_points) # Use only x, y, z for point cloud

    return pcd

lidar_pcd = read_lidar_data(lidar_bin)

o3d.visualization.draw_geometries([lidar_pcd])

In [ ]:
lidar_pcd = read_lidar_data(lidar_bin)
lidar_pcd

In [ ]:
lidar_pcd = o3d.geometry.PointCloud()
lidar_pcd.points = o3d.utility.Vector3dVector(get_velo2cam(lidar_bin)[:, :3])
lidar_pcd

# lidar_pcd.colors = o3d.utility.Vector3dVector(calculate_point_colors_based_on_distance(lidar_pcd))

In [ ]:
depth_pcd = depth_map_to_point_cloud(np.log(depth_map), K_left)
depth_pcd

In [ ]:
o3d.visualization.draw_geometries([
    lidar_pcd,
    # apply_rainbow_color(depth_pcd)
])

In [ ]:
vp = np.insert(np.fromfile(lidar_bin, dtype=np.float32).reshape((-1,4))[:,:3], 3, 1, axis=1).T
# vp = np.delete(vp, np.where(vp[]))

np.where(vp[3,:] < 0)

# print(vp)
# print(P_left)
# print(R_left_rect)
# print(T_cam_velo)

vc = P_left @ R_left_rect @ T_cam_velo @ vp
vc = np.delete(vc, np.where(vc[2,:] < 0)[0], axis=1)
vc[:2] /= vc[2, :]
 
# vc.shape
# vc.T.shape

# vf = vc.T

# T_velo_cam = np.linalg.inv(R_left_rect @ T_cam_velo)
# vf = T_velo_cam @ np.hstack((vf, np.ones((vf.shape[0], 1)))).T

# vf.shape

# vp = vf.T[:,:3]

vp = vc.T
vp.shape

o3d.geometry.Poin

In [ ]:
vpcd = o3d.geometry.PointCloud()
vpcd.points = o3d.utility.Vector3dVector(vp)
o3d.visualization.draw_geometries([vpcd])